In [1]:
import pandas as pd
import numpy as np
import requests
import swifter
import re

In [2]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')

In [3]:
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

# Preview dataframe
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artist
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,Rage Against The Machine
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,Rage Against The Machine
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,Rage Against The Machine
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,Rage Against The Machine
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,Rage Against The Machine


In [4]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# df['song'] = df['song'].str.replace(r"\(.*\)","")

df['song'] = df['song'].str.replace("\"", "") \
                        # .str.replace(",", "") \
                        # .str.replace("!", "")


# Create new column showing number of weeks in #1 spot (if exists)
df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist', 'rank'], as_index=False).count()['date']

# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,weeks-at-no1
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,1.0
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,7.0
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,1.0
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,4.0
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,2.0


In [5]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max',
                                                                                        'weeks-at-no1': 'max'})

# Create separate dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

# Display dataframe
unique_billboard_tracks_df.head()
unique_billboard_tracks_df

# df[(df['artist']=='Beastie Boys') & (df['year']==1986)]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
0,#1,Nelly,2001,22,11,NaN
1,#1,Nelly,2002,22,20,NaN
2,#1 Dee Jay,Goody Goody,1978,82,5,NaN
3,#9 Dream,John Lennon,1974,58,2,NaN
4,#9 Dream,John Lennon,1975,9,12,NaN
...,...,...,...,...,...,...
36078,whoknows,Musiq,2004,65,13,NaN
36079,www.memory,Alan Jackson,2000,45,9,NaN
36080,www.memory,Alan Jackson,2001,45,15,NaN
36081,¿Dònde Està Santa Claus? (Where Is Santa Claus?),Augie Rios,1958,47,3,NaN


In [6]:
# SPOTIFY API
# Import dependencies for Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret
from config import cid, secret


# Create objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
def search_spotify(song_title, artist):
    try:
        print(f"Trying '{song_title}' by {artist}")
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        track_id = searchResults['tracks']['items'][0]['id']
        print(f"{song_title} by {artist} was successful!")
        print(track_id)
        return track_id
    except:
        print(f"'{song_title}' by {artist} returned 0 results.")
    # return False

In [8]:
def adjust_parens(song_title):
    combinations = []

    strip_parens = ' '.join(song_title.strip(')(').split(')'))
    strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')
    combinations.append(strip_parens)

    try:
        inside_parens = re.findall(r'\(.*?\)', song_title)[0].strip('()').strip()
        
    except IndexError:
        pass
        
    try:
        left_parens = song_title.split(f"({inside_parens})")[0].strip()
        if (left_parens not in combinations) & (len(left_parens) > 0): 
            combinations.append(left_parens)
    except IndexError:
        left_parens = "NA"

    try:
        right_parens = song_title.split(f"({inside_parens})")[1].strip()
        if (right_parens not in combinations) & (len(right_parens) > 0): 
            combinations.append(right_parens)
    except IndexError:
        right_parens = "NA"

    return combinations

In [9]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    found_id = False
    
    while True:
        # Select first artist if multiple listed with "Featuring" keyword
        if 'Featuring' in artist:
            artist = artist.split(' Featuring ')[0]
        # Select first artist if multiple listed with "with" keyword
        elif ' with ' in artist:
            artist = artist.split(' with ')[0]
        elif ' With ' in artist:
            artist = artist.split(' With ')[0]
        elif "," in artist:
            artist = artist.split(',')[0]
        # Make API call
        found_id = search_spotify(song_title, artist)
        if isinstance(found_id, str):
            break

        # Search artist and song title (replacing words ending in "in" to "ing")
        song_title = re.sub(r"in\b", 'ing ', song_title)
        found_id = search_spotify(song_title, artist)
        if isinstance(found_id, str):
            break

        if '(' in song_title:
            for item in adjust_parens(song_title):
                found_id = search_spotify(item, artist)
                if isinstance(found_id, str):
                    break
        # Check for '/' character in song_title
        if '/' in song_title:
            # Try string on left side of '/'
            song_title = song_title.split('/')[0]
            found_id = search_spotify(song_title, artist)
            if isinstance(found_id, str):
                    break

            # Try string on right side of '/'
            try:
                song_title = song_title.split('/')[1]
                found_id = search_spotify(song_title, artist)
                if isinstance(found_id, str):
                    break
            except:
                pass

        # Check for parentheses in song_title and 
        song_title = ' '.join(song_title.strip(')(').split(')'))
        song_title = ' '.join(song_title.strip(' (').split('(')).replace('  ',' ')
        song_title

        # Check for '&' character
        if ' & ' in artist:
            artist = artist.split(' & ')[0]
            found_id = search_spotify(song_title, artist)
            if isinstance(found_id, str):
                break
        # Print song title and artist for non-match
        if isinstance(found_id, str):
            return found_id
    print(f"No ID found for '{song_title}' by {artist}")
    

In [10]:
def get_audio_features(id):
    searchResults = sp.audio_features(id)[0]
    return searchResults

In [11]:
song = 'You Only Live Once'
artist = "The Strokes"

result = sp.search(q=f"track:{song} artist:{artist} ", type="track")#['tracks']['items'][0]['id']

# result
# sp.search(q=f"track:{song} artist:{artist}", type='track', limit=1)
get_track_id(song, artist)
    # search_spotify(song, artist)
# len(result)

# adjust_parens(song)

Trying 'You Only Live Once' by The Strokes
You Only Live Once by The Strokes was successful!
4fPBB44eDH71YohayI4eKV
No ID found for 'You Only Live Once' by The Strokes


In [18]:
unique_billboard_tracks_df[unique_billboard_tracks_df['song']==song]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
20,(Baby) Turn On To Me,The Impressions,1970,56,8,NaN


In [19]:
# # Import dependency for swifter module


billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv', index=False)

In [35]:
# billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
# billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv', index=False)


billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv', index=False)


billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv', index=False)


billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv', index=False)


billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv', index=False)


billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv', index=False)

,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
0,1978,82,5,NaN,0MzHSJl9bHQiNPufYxJaab
1,1974,58,2,NaN,4ZVWvCUwsOnIGmJMj71RkG
2,1975,9,12,NaN,4ZVWvCUwsOnIGmJMj71RkG
3,1975,66,6,NaN,NaN
4,1978,79,4,NaN,3e7bMUM2jFwEYBgDqWCBDs
...,...,...,...,...,...
6246,1978,73,6,NaN,7jmU8QYtpfgDfi304r7klj
6247,1979,72,8,NaN,7jmU8QYtpfgDfi304r7klj
6248,1972,64,11,NaN,6QkdsyKK6Pq2wWuSgnpgQi
6249,1976,84,5,NaN,0aJHZYjwbfTmeyUWF7zGxI


In [56]:
# df_1960s[df_1960s['track_id']]

df_1970s = pd.read_csv('./Resources/billboard_1970s.csv')
null_rows = df_1970s[df_1970s['track_id'].isnull()]

# null_rows[null_rows['song'].str.contains("\?")]
# null_rows


null_rows['track_id'] = null_rows[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
null_rows.to_csv(f'./Resources/null1980s.csv', index=False)



In [28]:
missedvalues = pd.read_csv('./Resources/null1980s.csv')


missedvalues = missedvalues.drop(missedvalues.iloc[:,[0,1]], axis=1)

missedvalues.head()

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1,track_id
0,(A Ship Will Come) Ein Schiff Wird Kommen,Lale Anderson,1961,88,4,NaN,NaN
1,(Dance With The) Guitar Man,Duane Eddy and the Rebelettes,1962,12,13,NaN,NaN
2,(Dance With The) Guitar Man,Duane Eddy and the Rebelettes,1963,12,16,NaN,NaN
3,(Doin The) Lovers Leap,Webb Pierce,1960,93,2,NaN,NaN
4,(Hes) The Great Impostor,The Fleetwoods,1961,30,8,NaN,5S96nzXyrDBAwDNfgUwvWW
